# Augmentation Techniques from Scratch

In [98]:
import os
from PIL import Image
import numpy as np

## Defining the Techniques

### HSL Functions

In [99]:
def rgb_to_hsl(r, g, b):
    r, g, b = r / 255.0, g / 255.0, b / 255.0
    max_c = np.maximum(np.maximum(r, g), b)
    min_c = np.minimum(np.minimum(r, g), b)
    lightness = (max_c + min_c) / 2.0

    saturation = np.zeros_like(lightness)
    hue = np.zeros_like(lightness)

    mask = max_c != min_c
    dif = max_c - min_c
    saturation[mask] = np.where(lightness[mask] > 0.5, dif[mask] / (2.0 - max_c[mask] - min_c[mask]), dif[mask] / (max_c[mask] + min_c[mask]))

    mask_r = (max_c == r) & mask
    mask_g = (max_c == g) & mask
    mask_b = (max_c == b) & mask

    hue[mask_r] = ((g[mask_r] - b[mask_r]) / dif[mask_r] + (g[mask_r] < b[mask_r]) * 6) % 6
    hue[mask_g] = ((b[mask_g] - r[mask_g]) / dif[mask_g] + 2) % 6
    hue[mask_b] = ((r[mask_b] - g[mask_b]) / dif[mask_b] + 4) % 6
    hue /= 6

    return hue, saturation, lightness

def hsl_to_rgb(hue, saturation, lightness):
    def hue_to_rgb(p, q, t):
        t = np.where(t < 0, t + 1, t)
        t = np.where(t > 1, t - 1, t)
        return np.where(t < 1 / 6, p + (q - p) * 6 * t, 
                        np.where(t < 1 / 2, q,
                                 np.where(t < 2 / 3, p + (q - p) * (2 / 3 - t) * 6, p)))

    q = np.where(lightness < 0.5, lightness * (1 + saturation), lightness + saturation - lightness * saturation)
    p = 2 * lightness - q
    r = hue_to_rgb(p, q, hue + 1 / 3)
    g = hue_to_rgb(p, q, hue)
    b = hue_to_rgb(p, q, hue - 1 / 3)
    return (r * 255).astype(np.uint8), (g * 255).astype(np.uint8), (b * 255).astype(np.uint8)

### Saturation

In [100]:
def adjust_saturation(image, saturation_factor):
    r, g, b = image[..., 0], image[..., 1], image[..., 2]
    hue, saturation, lightness = rgb_to_hsl(r, g, b)
    saturation *= saturation_factor
    saturation = np.clip(saturation, 0, 1)

    r, g, b = hsl_to_rgb(hue, saturation, lightness)
    new_pixels = np.stack([r, g, b], axis=-1)

    new_image = Image.fromarray(new_pixels, 'RGB')
    return new_image

### Contrast

In [101]:
 
def adjust_contrast(pixels, factor):
    mean = np.mean(pixels)
    
    new_pixels = mean + (pixels - mean) * factor
    new_pixels = np.clip(new_pixels, 0, 255)
    
    new_pixels = new_pixels.astype(np.uint8)
    
    new_image = Image.fromarray(new_pixels)
    return new_image


### Brightness

In [102]:
def adjust_brightness(image, brightness_factor):
    r, g, b = image[..., 0], image[..., 1], image[..., 2]
    hue, saturation, lightness = rgb_to_hsl(r, g, b)
    lightness *= brightness_factor
    lightness = np.clip(lightness, 0, 1)

    r, g, b = hsl_to_rgb(hue, saturation, lightness)
    new_pixels = np.stack([r, g, b], axis=-1)

    new_image = Image.fromarray(new_pixels, 'RGB')
    return new_image

### Flipping

In [103]:
def flip_image(image, flip_type='horizontal'):
    if flip_type == 'horizontal':
        flipped_pixels = np.fliplr(image)
    elif flip_type == 'vertical':
        flipped_pixels = np.flipud(image)
    else:
        raise ValueError("Invalid flip_type. Choose from 'horizontal' or 'vertical'.")
    
    flipped_image = Image.fromarray(flipped_pixels)
    return flipped_image

### Rotation

In [104]:
def rotate_image(pixels, angle):
    angle_rad = np.deg2rad(angle)
    
    original_height, original_width, _ = pixels.shape

    center_x, center_y = original_width // 2, original_height // 2
    
    y_indices, x_indices = np.meshgrid(np.arange(original_height), np.arange(original_width), indexing='ij')
    
    x_indices_flat = x_indices.flatten()
    y_indices_flat = y_indices.flatten()
    
    relative_x = x_indices_flat - center_x
    relative_y = y_indices_flat - center_y
    
    orig_x_flat = (relative_x * np.cos(angle_rad) + relative_y * np.sin(angle_rad)).astype(int) + center_x
    orig_y_flat = (-relative_x * np.sin(angle_rad) + relative_y * np.cos(angle_rad)).astype(int) + center_y
    valid_mask = (orig_x_flat >= 0) & (orig_x_flat < original_width) & (orig_y_flat >= 0) & (orig_y_flat < original_height)
    
    rotated_image = np.zeros_like(pixels)
    rotated_image[y_indices_flat[valid_mask], x_indices_flat[valid_mask]] = pixels[orig_y_flat[valid_mask], orig_x_flat[valid_mask]]
    
    new_image = Image.fromarray(rotated_image)

    return new_image

### Translation

In [105]:
def translate_image(pixels, tx, ty):
    original_height, original_width, _ = pixels.shape

    translated_image = np.zeros_like(pixels)

    y_indices, x_indices = np.meshgrid(np.arange(original_height), np.arange(original_width), indexing='ij')

    translated_x = x_indices - tx
    translated_y = y_indices - ty

    valid_mask = (translated_x >= 0) & (translated_x < original_width) & (translated_y >= 0) & (translated_y < original_height)

    translated_image[y_indices[valid_mask], x_indices[valid_mask]] = pixels[translated_y[valid_mask], translated_x[valid_mask]]
    new_image = Image.fromarray(translated_image)

    return new_image

## Applying Transformations

In [106]:
def save_image(image, original_name, suffix, output_dir):
    base_name, ext = os.path.splitext(original_name)
    new_name = f"{base_name}_{suffix}{ext}"
    image.save(os.path.join(output_dir, new_name))

input_dir = '../images'
output_dir = 'augmented_images'

image_files = [f for f in os.listdir(input_dir) if f.endswith(('.jpeg'))]

for image_file in image_files:
    image_path = os.path.join(input_dir, image_file)
    image = Image.open(image_path)
    pixels = np.array(image)
    transformed_images = {}

    # Transformations
    transformed_images['saturation_0.5'] = adjust_saturation(pixels, 0.5)
    transformed_images['saturation_1.5'] = adjust_saturation(pixels, 1.5)
    transformed_images['flipped_horizontal'] = flip_image(pixels, 'horizontal')
    transformed_images['flipped_vertical'] = flip_image(pixels, 'vertical')
    transformed_images['contrast_0.5'] = adjust_contrast(pixels, 0.5)
    transformed_images['contrast_2.0'] = adjust_contrast(pixels, 2.0)
    transformed_images['translated_10_20'] = translate_image(pixels, 10, 20)
    transformed_images['translated_30_50'] = translate_image(pixels, 30, 50)
    transformed_images['brightness_0.5'] = adjust_brightness(pixels, 0.5)
    transformed_images['brightness_2.0'] = adjust_brightness(pixels, 2.0)
    transformed_images['rotated_90'] = rotate_image(pixels, 90)
    transformed_images['rotated_180'] = rotate_image(pixels, 180)

    for suffix, img in transformed_images.items():
        save_image(img, image_file, suffix, output_dir)